In [1]:
import pandas as pd
from useful_functions import * 

from dpks.quant_matrix import QuantMatrix

In [4]:
df = pd.read_csv('../data/220316_ghost_nrt_filtered.tsv', sep='\t')
design_matrix = pd.read_csv('../data/design_matrix_group_de_early_time_points_subtypes.tsv', sep='\t')

GroupOneCols = design_matrix[design_matrix['Group'] == 1]['Sample'].values
GroupTwoCols = design_matrix[design_matrix['Group'] == 2]['Sample'].values

quant_matrix = QuantMatrix(
    quantification_file="../data/220316_ghost_nrt_filtered.tsv",
    design_matrix_file="../data/design_matrix_group_de_early_time_points_subtypes.tsv"
)


quantified_proteins = (
    quant_matrix.
    filter()
    .normalize(method="mean")
    .quantify(method="maxlfq")
).to_df()

In [5]:
from sklearn.model_selection import train_test_split

df1 = quantified_proteins[GroupOneCols].T
df2 = quantified_proteins[GroupTwoCols].T

y = [1 for x in GroupOneCols]
y += [2 for x in GroupTwoCols]
X = pd.concat([df1,df2])
X = X.fillna(0).to_numpy()

print(f"Number of features: {len(X[1])}")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f'Training datapoints: {len(y_train)}. Testing datapoints {len(y_test)}')


Number of features: 1039
Training datapoints: 57. Testing datapoints 15


In [9]:
from sklearn import preprocessing

# I think there is a problem with scaling since the data is sparse

scaler = preprocessing.StandardScaler(with_mean=False).fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)



array([[32.27775943, 33.90685418, 32.01153753, ...,  0.        ,
         0.        ,  0.        ],
       [31.76490117, 35.08530593, 31.33449548, ...,  0.        ,
         0.        ,  0.        ],
       [31.28394312, 34.97840528, 30.94981859, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [31.33805864, 34.17679537, 31.07706221, ...,  0.        ,
         0.        ,  0.        ],
       [29.7145224 , 34.51796941, 29.52255312, ...,  0.        ,
         3.49796962,  2.32938072],
       [31.31994867, 34.97560667, 29.7619668 , ...,  0.        ,
         0.        ,  0.        ]])

In [7]:
# SVM
from sklearn import svm

clf = svm.SVC()
clf.fit(X_train, y_train)
clf.predict(X_test)

array([2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])